# Analysis

## ToDo

- Importing more than one data json
- Handling more subjects in the analysis
- Model specific confusion rate
- How to manage or represent all this data
- ...

## Datastructure

## Code

In [27]:
import numpy as np
import pandas as pd
import json
with open('data.json','r') as f:
    data = json.loads(f.read())

In [28]:
df = pd.json_normalize(data)

df = df[df['test_part'] == 'response'].reset_index()

df['button_pressed'] = df['button_pressed'].astype(int)
df['test_stimulus.position'] = df['test_stimulus.position'].astype(int)

df = df.rename(columns={"test_stimulus.left": "left",
                        "test_stimulus.right": "right",
                        "test_stimulus.att_check": "att_check",
                        "test_stimulus.position": "position",
                        "button_pressed":"pressed"})

In [29]:
df = df[['subject','rt','pressed','position',
    'left','right','att_check']]

In [30]:
df

,subject,rt,pressed,position,left,right,att_check
0,r6vg0soyf6xlcpt,852.0,1,1,bvh/vr_prediction_models/model(vcgpdm)-dataset...,bvh/return-bottle-hold-training2-lines.txt,False
1,r6vg0soyf6xlcpt,822.0,1,1,bvh/vr_prediction_models/model(dmp)-dataset(re...,bvh/return-bottle-training2-lines.txt,True
2,r6vg0soyf6xlcpt,1167.0,1,1,bvh/vr_prediction_models/model(dmp)-dataset(pa...,bvh/pass-bottle-training1-lines.txt,False
3,r6vg0soyf6xlcpt,855.0,1,1,bvh/vr_prediction_models/model(vcgpdm)-dataset...,bvh/return-bottle-hold-training3-lines.txt,False
4,r6vg0soyf6xlcpt,935.0,1,0,bvh/pass-bottle-hold-training0-lines.txt,bvh/vr_prediction_models/model(vcgpdm)-dataset...,False
...,...,...,...,...,...,...,...
105,r6vg0soyf6xlcpt,710.0,0,0,bvh/return-bottle-training2-lines.txt,bvh/vr_prediction_models/model(vcgpdm)-dataset...,False
106,r6vg0soyf6xlcpt,1247.0,1,1,bvh/vr_prediction_models/model(dmp)-dataset(pa...,bvh/pass-bottle-hold-training3-lines.txt,False
107,r6vg0soyf6xlcpt,1184.0,1,0,bvh/return-bottle-training3-lines.txt,bvh/vr_prediction_models/model(vcgpdm)-dataset...,False
108,r6vg0soyf6xlcpt,966.0,0,0,bvh/return-bottle-hold-training1-lines.txt,bvh/vr_prediction_models/model(dmp)-dataset(re...,False


In [31]:
d = df.copy()

In [32]:
d['left'] = d['left'].str.replace('bvh/vr_prediction_models/','')
d['left'] = d['left'].str.replace('bvh/','')
d['left'] = d['left'].str.replace('/final-lines.txt','')
d['left'] = d['left'].str.replace('-lines.txt','')
d['right'] = d['right'].str.replace('bvh/vr_prediction_models/','')
d['right'] = d['right'].str.replace('bvh/','')
d['right'] = d['right'].str.replace('/final-lines.txt','')
d['right'] = d['right'].str.replace('-lines.txt','')

In [33]:
d

,subject,rt,pressed,position,left,right,att_check
0,r6vg0soyf6xlcpt,852.0,1,1,model(vcgpdm)-dataset(return-bottle-hold)-mode...,return-bottle-hold-training2,False
1,r6vg0soyf6xlcpt,822.0,1,1,model(dmp)-dataset(return-bottle)-npsi(90)-hol...,return-bottle-training2,True
2,r6vg0soyf6xlcpt,1167.0,1,1,model(dmp)-dataset(pass-bottle)-npsi(70)-hold(1),pass-bottle-training1,False
3,r6vg0soyf6xlcpt,855.0,1,1,model(vcgpdm)-dataset(return-bottle-hold)-mode...,return-bottle-hold-training3,False
4,r6vg0soyf6xlcpt,935.0,1,0,pass-bottle-hold-training0,model(vcgpdm)-dataset(pass-bottle-hold)-mode(E...,False
...,...,...,...,...,...,...,...
105,r6vg0soyf6xlcpt,710.0,0,0,return-bottle-training2,model(vcgpdm)-dataset(return-bottle)-mode(ELBO...,False
106,r6vg0soyf6xlcpt,1247.0,1,1,model(dmp)-dataset(pass-bottle-hold)-npsi(20)-...,pass-bottle-hold-training3,False
107,r6vg0soyf6xlcpt,1184.0,1,0,return-bottle-training3,model(vcgpdm)-dataset(return-bottle)-mode(ELBO...,False
108,r6vg0soyf6xlcpt,966.0,0,0,return-bottle-hold-training1,model(dmp)-dataset(return-bottle-hold)-npsi(50...,False


## Individual confusion rate over all models

### with all attention checks included

In [50]:
confusion = []
for i in zip(d['pressed'],d['position']):
    confusion.append(i)
right = 0
for i in confusion:
    if i[0] == i[1]:
        right += 1
confusion_rate = (len(confusion) - right)/len(confusion)
print(confusion_rate)

0.33636363636363636


### without attention checks

In [54]:
no_check = d[d['att_check']==False]

In [55]:
no_check

,subject,rt,pressed,position,left,right,att_check
0,r6vg0soyf6xlcpt,852.0,1,1,model(vcgpdm)-dataset(return-bottle-hold)-mode...,return-bottle-hold-training2,False
2,r6vg0soyf6xlcpt,1167.0,1,1,model(dmp)-dataset(pass-bottle)-npsi(70)-hold(1),pass-bottle-training1,False
3,r6vg0soyf6xlcpt,855.0,1,1,model(vcgpdm)-dataset(return-bottle-hold)-mode...,return-bottle-hold-training3,False
4,r6vg0soyf6xlcpt,935.0,1,0,pass-bottle-hold-training0,model(vcgpdm)-dataset(pass-bottle-hold)-mode(E...,False
5,r6vg0soyf6xlcpt,867.0,1,0,return-bottle-hold-training3,model(vcgpdm)-dataset(return-bottle-hold)-mode...,False
...,...,...,...,...,...,...,...
105,r6vg0soyf6xlcpt,710.0,0,0,return-bottle-training2,model(vcgpdm)-dataset(return-bottle)-mode(ELBO...,False
106,r6vg0soyf6xlcpt,1247.0,1,1,model(dmp)-dataset(pass-bottle-hold)-npsi(20)-...,pass-bottle-hold-training3,False
107,r6vg0soyf6xlcpt,1184.0,1,0,return-bottle-training3,model(vcgpdm)-dataset(return-bottle)-mode(ELBO...,False
108,r6vg0soyf6xlcpt,966.0,0,0,return-bottle-hold-training1,model(dmp)-dataset(return-bottle-hold)-npsi(50...,False


In [61]:
confusion = []
for i in zip(no_check['pressed'],no_check['position']):
    confusion.append(i)
right = 0
for i in confusion:
    if i[0] == i[1]:
        right += 1
confusion_rate = (len(confusion) - right)/len(confusion)
print(confusion_rate)

0.37


## Checking attention
### if confusion rate is zero -> good

In [59]:
check_att = d[d['att_check']==True]

In [60]:
confusion = []
for i in zip(check_att['pressed'],check_att['position']):
    confusion.append(i)
right = 0
for i in confusion:
    if i[0] == i[1]:
        right += 1
confusion_rate = (len(confusion) - right)/len(confusion)
print(confusion_rate)

0.0
